In [1]:
# Install dependencies
!pip install pandas numpy folium

import pandas as pd
import numpy as np
import folium
from folium.plugins import HeatMap

# -----------------------------
# 1. All-India city base layer
# -----------------------------
cities_data = [
    ("Delhi", 28.6139, 77.2090),
    ("Mumbai", 19.0760, 72.8777),
    ("Kolkata", 22.5726, 88.3639),
    ("Chennai", 13.0827, 80.2707),
    ("Hyderabad", 17.3850, 78.4867),
    ("Bengaluru", 12.9716, 77.5946),
    ("Ahmedabad", 23.0225, 72.5714),
    ("Pune", 18.5204, 73.8567),
    ("Jaipur", 26.9124, 75.7873),
    ("Lucknow", 26.8467, 80.9462),
    ("Kanpur", 26.4499, 80.3319),
    ("Varanasi", 25.3176, 82.9739),
    ("Patna", 25.5941, 85.1376),
    ("Ranchi", 23.3441, 85.3096),
    ("Raipur", 21.2514, 81.6296),
    ("Bhubaneswar", 20.2961, 85.8245),
    ("Guwahati", 26.1445, 91.7362),
    ("Chandigarh", 30.7333, 76.7794),
    ("Ludhiana", 30.9010, 75.8573),
    ("Amritsar", 31.6340, 74.8723),
    ("Srinagar", 34.0837, 74.7973),
    ("Jammu", 32.7266, 74.8570),
    ("Thiruvananthapuram", 8.5241, 76.9366),
    ("Kochi", 9.9312, 76.2673),
    ("Visakhapatnam", 17.6868, 83.2185),
    ("Vijayawada", 16.5062, 80.6480),
    ("Mysuru", 12.2958, 76.6394),
    ("Noida", 28.5355, 77.3910),
    ("Gurugram", 28.4595, 77.0266),
    ("Surat", 21.1702, 72.8311),
    ("Vadodara", 22.3072, 73.1812),
    ("Indore", 22.7196, 75.8577),
    ("Bhopal", 23.2599, 77.4126),
    ("Nagpur", 21.1458, 79.0882),
    ("Coimbatore", 11.0168, 76.9558),
    ("Madurai", 9.9252, 78.1198),
    ("Mangaluru", 12.9141, 74.8560),
    ("Udaipur", 24.5854, 73.7125),
    ("Jodhpur", 26.2389, 73.0243),
    ("Agra", 27.1767, 78.0081),
    ("Meerut", 28.9845, 77.7064),
    ("Prayagraj", 25.4358, 81.8463),
]

cities_df = pd.DataFrame(cities_data, columns=["city", "lat", "lon"])

# -----------------------------------------
# 2. Generate synthetic GeoHealth dataset
# -----------------------------------------
np.random.seed(42)

n = len(cities_df)

# Air Quality Index (higher is worse)
aqi = np.random.randint(60, 420, size=n)

# Heat index (deg C, higher is worse)
heat_index = np.random.uniform(26.0, 45.0, size=n)

# Water Quality Index (higher is better, so we will invert for risk)
water_quality = np.random.randint(40, 95, size=n)

# Population density (people per sq. km)
population_density = np.random.randint(1000, 35000, size=n)

geohealth = cities_df.copy()
geohealth["AQI"] = aqi
geohealth["heat_index"] = heat_index
geohealth["water_quality_index"] = water_quality
geohealth["population_density"] = population_density

# -----------------------------------------
# 3. Normalization and health risk scoring
# -----------------------------------------
def normalize(series):
    if series.max() == series.min():
        return series * 0
    return (series - series.min()) / (series.max() - series.min())

geohealth["aqi_norm"] = normalize(geohealth["AQI"])
geohealth["heat_norm"] = normalize(geohealth["heat_index"])

# Water quality: lower is worse, so invert
wq_norm_raw = normalize(geohealth["water_quality_index"])
geohealth["water_norm"] = 1.0 - wq_norm_raw

geohealth["pop_norm"] = normalize(geohealth["population_density"])

# Final health risk score
geohealth["health_risk_score"] = (
    geohealth["aqi_norm"] * 0.40 +
    geohealth["heat_norm"] * 0.20 +
    geohealth["water_norm"] * 0.20 +
    geohealth["pop_norm"] * 0.20
)

# Risk level classification
def label_risk(x):
    if x >= 0.80:
        return "Severe"
    elif x >= 0.60:
        return "High"
    elif x >= 0.40:
        return "Moderate"
    else:
        return "Low"

geohealth["risk_level"] = geohealth["health_risk_score"].apply(label_risk)

# Save dataset
geohealth.to_csv("geohealth_dataset.csv", index=False)
print("Saved: geohealth_dataset.csv")

# -----------------------------------------
# 4. Build GeoHealth dashboard map
# -----------------------------------------
center_lat = geohealth["lat"].mean()
center_lon = geohealth["lon"].mean()

m = folium.Map(location=[center_lat, center_lon], zoom_start=5, tiles="CartoDB positron")

# Heatmap layer using health risk score
heat_data = [
    [row["lat"], row["lon"], row["health_risk_score"]]
    for _, row in geohealth.iterrows()
]
HeatMap(heat_data, radius=30, blur=20, min_opacity=0.3).add_to(m)

# Color for risk levels
def risk_color(level):
    if level == "Severe":
        return "darkred"
    elif level == "High":
        return "red"
    elif level == "Moderate":
        return "orange"
    else:
        return "green"

# Add city markers
for _, row in geohealth.iterrows():
    popup_text = (
        f"City: {row['city']}<br>"
        f"AQI: {row['AQI']}<br>"
        f"Heat Index: {row['heat_index']:.1f} C<br>"
        f"Water Quality Index: {row['water_quality_index']}<br>"
        f"Population Density: {row['population_density']} per sq.km<br>"
        f"Health Risk Score: {row['health_risk_score']:.2f}<br>"
        f"Risk Level: {row['risk_level']}"
    )
    folium.CircleMarker(
        location=[row["lat"], row["lon"]],
        radius=7,
        weight=1,
        color=risk_color(row["risk_level"]),
        fill=True,
        fill_opacity=0.9,
        popup=folium.Popup(popup_text, max_width=260)
    ).add_to(m)

dashboard_path = "geohealth_dashboard.html"
m.save(dashboard_path)
print("Saved:", dashboard_path)

# -----------------------------------------
# 5. Text summary report
# -----------------------------------------
sorted_geo = geohealth.sort_values("health_risk_score", ascending=False)

with open("geohealth_report.txt", "w") as f:
    f.write("AetherVision Omega - City GeoHealth Summary\n")
    f.write("===========================================\n\n")
    for _, row in sorted_geo.iterrows():
        f.write(f"City: {row['city']}\n")
        f.write(f"  Health Risk Score: {row['health_risk_score']:.2f}\n")
        f.write(f"  Risk Level: {row['risk_level']}\n")
        f.write(f"  AQI: {row['AQI']}\n")
        f.write(f"  Heat Index: {row['heat_index']:.1f} C\n")
        f.write(f"  Water Quality Index: {row['water_quality_index']}\n")
        f.write(f"  Population Density: {row['population_density']} per sq.km\n")
        f.write("\n")

print("Saved: geohealth_report.txt")

Saved: geohealth_dataset.csv
Saved: geohealth_dashboard.html
Saved: geohealth_report.txt


In [2]:
!zip -r geohealth_outputs.zip geohealth_dashboard.html geohealth_dataset.csv geohealth_report.txt

  adding: geohealth_dashboard.html (deflated 85%)
  adding: geohealth_dataset.csv (deflated 51%)
  adding: geohealth_report.txt (deflated 83%)
